In [ ]:
api_key = 'your api key'

# 文档读取

In [ ]:
from langchain_community.document_loaders import PyMuPDFLoader

pdf_path = 'path/to/大模型基础 完整版.pdf'
loader = PyMuPDFLoader(pdf_path)
pdf_pages = loader.load()


# 文档分割

In [9]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap = 50)
split_docs = text_splitter.split_documents(pdf_pages)
print(f"切分后的文件数量：{len(split_docs)}")


切分后的文件数量：652


# 存入向量数据库

In [ ]:
from zhipuai_embedding import ZhipuAIEmbeddings
from langchain_community.vectorstores import Chroma
embedding = ZhipuAIEmbeddings()
persist_directory = 'path/to/myAgent/mydb'
vectordb = Chroma.from_documents(
    embedding=embedding,
    persist_directory=persist_directory,
    documents=split_docs
)


# 读取数据库

In [11]:
vectordb = Chroma(
    embedding_function=embedding,
    persist_directory=persist_directory
)


# 初始化模型

In [12]:
from zhipuai_llm import ZhipuaiLLM
zhipuai_model = ZhipuaiLLM(model_name="glm-4-plus", temperature=0.1, api_key=api_key)

In [13]:
zhipuai_model.invoke('你好')

AIMessage(content='你好👋！我是人工智能助手智谱清言（ChatGLM），很高兴见到你，欢迎问我任何问题。', additional_kwargs={}, response_metadata={'time_in_seconds': 0.955}, id='run-7d11b76e-ee6e-47c5-b066-a01de92cd409-0', usage_metadata={'input_tokens': 6, 'output_tokens': 28, 'total_tokens': 34})

# 构建检索问答链

In [14]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
template = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
案。最多使用三句话。尽量使答案简明扼要。请你在回答的最后说“谢谢你的提问！”。
{context}
问题: {input}
"""

retriveal_chain = vectordb.as_retriever()

prompt = PromptTemplate(template=template)
qa_chain = (
    RunnableParallel({'context':retriveal_chain, 'input':RunnablePassthrough()})
    | prompt
    | zhipuai_model
    | StrOutputParser()
)

In [15]:
qa_chain.invoke('什么是prompt工程？')

'Prompt工程是一种专注于编写输入指令（Prompt）的技术，用于指导生成式人工智能模型执行特定任务，而无需进行繁琐的微调。它通过精心设计的Prompt引导大模型直接适应下游任务，提高模型性能和效率。谢谢你的提问！'

# 带历史记忆的多轮对话

In [16]:
from langchain_core.messages import AIMessage
from langchain_core.messages import HumanMessage
# 传入历史记录，重新让模型计算
zhipuai_model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)


AIMessage(content='Your name is Bob! How can I help you today, Bob?', additional_kwargs={}, response_metadata={'time_in_seconds': 0.81}, id='run-2aee8292-91dd-4a1e-97e4-3b9687a287dd-0', usage_metadata={'input_tokens': 29, 'output_tokens': 16, 'total_tokens': 45})

In [17]:
# 添加链
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
chain = prompt | zhipuai_model


In [18]:
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory
# 存储历史记忆
store = {}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(chain, get_session_history)


In [19]:
config = {"configurable": {"session_id": "abc5"}}
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Jim")],
    config=config,
)

response.content


'Hello Jim! How can I assist you today? If you have any questions or need help with something, feel free to let me know!'

In [20]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Jim! How can I help you today, Jim?'

# 多个输入的多轮对话

In [21]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | zhipuai_model


In [22]:
# 指定那个key是输入
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config = {"configurable": {"session_id": "abc11"}}


In [23]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="hi! I'm todd")], "language": "Spanish"},
    config=config,
)

response.content


'¡Hola, Todd! ¿En qué puedo ayudarte hoy?'

In [24]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Spanish"},
    config=config,
)

response.content


'Tu nombre es Todd. ¿Hay algo más en lo que te pueda ayudar?'

# 检索+多轮对话

In [ ]:
from langchain_community.vectorstores import Chroma
import os
from operator import itemgetter
from typing import List
from langchain_core.documents import Document

# 构建检索器
persist_directory = 'path/to/myAgent/mydb'
vectordb = Chroma(
    embedding_function=embedding,
    persist_directory=persist_directory
)
retriever = vectordb.as_retriever(search_kwargs={"k": 4})
def format_docs(docs: List[Document]) -> str:
    # 将检索到的文档拼成字符串，便于塞入 {context}
    return "\n\n".join(f"[{i+1}] {d.page_content}" for i, d in enumerate(docs))


In [26]:
# 历史记忆
session_store = {}

def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in session_store:
        session_store[session_id] = InMemoryChatMessageHistory()
    return session_store[session_id]

In [27]:
# prompt
contextual_prompt = ChatPromptTemplate.from_messages([
    ("system", "你是资深推理系统工程师。请综合【知识库】与对话历史，给出准确、简洁、条理清晰的回答。\n【知识库】\n{context}"),
    MessagesPlaceholder(variable_name="history"),   # 关键：用于注入历史消息
    ("human", "当前问题：{input}")
])

In [28]:
# rag
base_rag_chain = (
    {
        # 仅把用户 query 传给检索器，然后把文档格式化为字符串
        "context": itemgetter("input") | retriever | format_docs,
        # 提示模板里需要 {input}
        "input": itemgetter("input"),
        # 一定要把 history 这个键继续向下传，供 MessagesPlaceholder 使用
        "history": itemgetter("history"),
    }
    | contextual_prompt
    | zhipuai_model
)

In [29]:
# 历史记忆
conversational_chain = RunnableWithMessageHistory(
    base_rag_chain,
    get_session_history,
    input_messages_key="input",     # 指明“用户输入”的键
    history_messages_key="history", # 必须与 MessagesPlaceholder 的 variable_name 一致
)
config = {"configurable": {"session_id": "user_123"}}


In [30]:
response1 = conversational_chain.invoke(
    {"input": "提示词工程是什么？"},
    config=config
)

In [31]:
print(response1.content)

提示词工程（Prompt Engineering）是一种专注于如何编写和设计输入指令（Prompt）的技术，用于指导生成式人工智能模型执行特定任务。这些指令通常以自然语言文本形式出现，核心目的是清晰地描述模型应执行的任务，以引导模型生成特定的文本、图像、音频等内容。

传统的自然语言处理研究遵循“预训练-微调-预测”范式，而提示词工程则属于“预训练-提示预测”新范式的一部分。在这种新范式下，通过精心设计的Prompt，预训练模型可以直接适应下游任务，无需繁琐的微调过程。Prompt的设计对模型性能有深远影响，是自然语言处理领域中的重要技术。


In [32]:
response1 = conversational_chain.invoke(
    {"input": "它有哪些技巧"},
    config=config
)
print(response1.content)

提示词工程（Prompt Engineering）涉及多种技巧，以优化生成式人工智能模型的输出。以下是一些常见的技巧：

### 1. **明确任务指令**
- **具体化**：明确指出模型需要完成的任务，避免模糊不清的描述。
- **示例引导**：提供具体的示例，帮助模型理解期望的输出格式。

### 2. **上下文提供**
- **背景信息**：提供相关的背景信息，帮助模型更好地理解任务。
- **相关数据**：包含相关的数据或事实，增强模型的生成依据。

### 3. **格式规范**
- **结构化提示**：使用结构化的格式，如列表、表格等，使提示更易理解。
- **模板化**：设计标准化的模板，确保输出的一致性。

### 4. **逐步引导**
- **分步提示**：将复杂任务分解为多个步骤，逐步引导模型。
- **中间步骤**：提供中间步骤的示例，帮助模型逐步逼近最终目标。

### 5. **多样化表达**
- **同义词使用**：使用不同的词汇表达相同的意思，增加模型的灵活性。
- **多角度描述**：从不同角度描述任务，丰富模型的输入信息。

### 6. **情感与语气**
- **情感引导**：根据任务需求，调整提示中的情感倾向。
- **语气调整**：使用适当的语气，如正式、幽默等，以符合输出风格。

### 7. **长度控制**
- **简洁性**：尽量保持提示简洁，避免冗长信息干扰。
- **信息量平衡**：确保提示包含足够的信息，但不过度冗余。

### 8. **反馈与迭代**
- **结果反馈**：根据模型的输出，调整和优化提示。
- **迭代优化**：多次迭代，逐步改进提示的效果。

### 9. **避免歧义**
- **清晰表达**：避免使用容易引起歧义的词汇或表达。
- **明确指代**：确保提示中的指代关系明确，避免混淆。

### 10. **利用先验知识**
- **领域知识**：结合特定领域的知识，提升模型的专业性。
- **常识引入**：引入常识性信息，帮助模型做出更合理的推断。

通过综合运用这些技巧，可以有效提升生成式模型的输出质量和任务适应性。
